In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/transacciones_logistica_v2.csv')

In [2]:
df.columns

Index(['Transaccion_ID', 'SKU_ID', 'Fecha_Venta', 'Cantidad_Vendida',
       'Precio_Venta_Final', 'Costo_Envio', 'Tiempo_Entrega_Real',
       'Estado_Envio', 'Ciudad_Destino', 'Canal_Venta'],
      dtype='object')

In [3]:
Transaccion_ID = 'Transaccion_ID'
SKUid = 'SKU_ID'
Fecha_venta = 'Fecha_Venta'
Cantidad_vendida = 'Cantidad_Vendida'
Precio_venta_final = 'Precio_Venta_Final'
Costo_envio = 'Costo_Envio'
Tiempo_entrega_real = 'Tiempo_Entrega_Real'
Estado_envio = 'Estado_Envio'
Ciudad_destino = 'Ciudad_Destino'
Canal_venta = 'Canal_Venta'


In [5]:
df[Transaccion_ID].describe()

count         10000
unique        10000
top       TRX-10000
freq              1
Name: Transaccion_ID, dtype: object

In [7]:
df[Transaccion_ID].isna().sum()
df[Transaccion_ID].isnull().sum()

np.int64(0)

In [8]:
df[SKUid].describe()

count         10000
unique         2889
top       PROD-1229
freq             11
Name: SKU_ID, dtype: object

In [10]:
df[SKUid].isnull().sum()


np.int64(0)

In [16]:
df[Tiempo_entrega_real].describe()

count    10000.000000
mean        19.881900
std         69.914879
min          1.000000
25%          8.000000
50%         15.000000
75%         22.000000
max        999.000000
Name: Tiempo_Entrega_Real, dtype: float64

In [27]:
df[Cantidad_vendida].describe()

count    10000.000000
mean         7.413600
std          4.205485
min         -5.000000
25%          4.000000
50%          7.000000
75%         11.000000
max         14.000000
Name: Cantidad_Vendida, dtype: float64

In [28]:
df[Costo_envio].describe()

count    9166.000000
mean       52.411765
std        27.419908
min         5.000000
25%        28.802500
50%        52.360000
75%        76.110000
max        99.990000
Name: Costo_Envio, dtype: float64

In [29]:
df[Costo_envio].isna().sum()

np.int64(834)

In [31]:
df[Precio_venta_final].describe()

count    10000.000000
mean       996.392443
std        570.571003
min         10.010000
25%        500.265000
50%        999.230000
75%       1479.225000
max       1999.580000
Name: Precio_Venta_Final, dtype: float64

In [39]:
print(df[Ciudad_destino].describe())
print(df[Ciudad_destino].isna().sum())
print(df[Ciudad_destino].value_counts())


count          10000
unique             8
top       Ventas_Web
freq            1290
Name: Ciudad_Destino, dtype: object
0
Ciudad_Destino
Ventas_Web      1290
BOG             1267
Bogotá          1261
Cali            1256
Bucaramanga     1250
Medellín        1234
MED             1223
Barranquilla    1219
Name: count, dtype: int64


In [40]:
# Reemplazamos las abreviaturas de ciudades por nombres completos
df[Ciudad_destino] = df[Ciudad_destino].replace({'BOG': 'Bogotá', 'MED': 'Medellín'})
print("Valores después del reemplazo:")
print(df[Ciudad_destino].value_counts())

Valores después del reemplazo:
Ciudad_Destino
Bogotá          2528
Medellín        2457
Ventas_Web      1290
Cali            1256
Bucaramanga     1250
Barranquilla    1219
Name: count, dtype: int64


In [43]:
df[Canal_venta].describe()
df[Canal_venta].value_counts()

Canal_Venta
Físico      2532
Online      2518
WhatsApp    2510
App         2440
Name: count, dtype: int64